In [116]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
import pickle

In [117]:
data=pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [118]:
##data.head() shows that RowNumber,CustomerId,Surname are not useful for prediction
X=data.drop(['RowNumber','CustomerId','Surname','Exited'],axis=1)
y=data['Exited']
print(X.head())
print(y.head())

   CreditScore Geography  Gender  Age  Tenure    Balance  NumOfProducts  \
0          619    France  Female   42       2       0.00              1   
1          608     Spain  Female   41       1   83807.86              1   
2          502    France  Female   42       8  159660.80              3   
3          699    France  Female   39       1       0.00              2   
4          850     Spain  Female   43       2  125510.82              1   

   HasCrCard  IsActiveMember  EstimatedSalary  
0          1               1        101348.88  
1          0               1        112542.58  
2          1               0        113931.57  
3          0               0         93826.63  
4          1               1         79084.10  
0    1
1    0
2    1
3    0
4    0
Name: Exited, dtype: int64


In [119]:
X.info()    
y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10000 non-null  int64  
 1   Geography        10000 non-null  object 
 2   Gender           10000 non-null  object 
 3   Age              10000 non-null  int64  
 4   Tenure           10000 non-null  int64  
 5   Balance          10000 non-null  float64
 6   NumOfProducts    10000 non-null  int64  
 7   HasCrCard        10000 non-null  int64  
 8   IsActiveMember   10000 non-null  int64  
 9   EstimatedSalary  10000 non-null  float64
dtypes: float64(2), int64(6), object(2)
memory usage: 781.4+ KB
<class 'pandas.core.series.Series'>
RangeIndex: 10000 entries, 0 to 9999
Series name: Exited
Non-Null Count  Dtype
--------------  -----
10000 non-null  int64
dtypes: int64(1)
memory usage: 78.3 KB


In [120]:
cat_features = [col for col in X.columns if X[col].dtype == 'object']
num_features = [col for col in X.columns if X[col].dtype != 'object']
print(cat_features)
print(num_features)

['Geography', 'Gender']
['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary']


In [121]:
data.nunique()

RowNumber          10000
CustomerId         10000
Surname             2932
CreditScore          460
Geography              3
Gender                 2
Age                   70
Tenure                11
Balance             6382
NumOfProducts          4
HasCrCard              2
IsActiveMember         2
EstimatedSalary     9999
Exited                 2
dtype: int64

In [122]:
#Geography and Gender are categorical features
#each having 3 and 2 unique values respectively.
labEnco=LabelEncoder()
X['Gender'] = labEnco.fit_transform(X['Gender'])
#X['Geography'] = labEnco.fit_transform(X['Geography'])
X[cat_features]

,Geography,Gender
0,France,0
1,Spain,0
2,France,0
3,France,0
4,Spain,0
...,...,...
9995,France,1
9996,France,1
9997,France,0
9998,Germany,1


In [123]:
#Since ANN is all about calculations thus somehow France<Spain<Germany weight wise 
#wouldn't affect much if we use LabelEncoder for Gender
#thus we will use OHE and not LabelEncoder for Location
#OHE MANUAL METHOD
OHE = OneHotEncoder()
geo_encoded= OHE.fit_transform(X[['Geography']]).toarray() # -> we have created a sparse matrix of geo_encoding
geo_cols = OHE.get_feature_names_out(['Geography']) # -> ['Geography_France' 'Geography_Germany' 'Geography_Spain']
geo_df = pd.DataFrame(geo_encoded, columns=geo_cols) # -> creating a DataFrame for the encoded geography
X = pd.concat([X.drop('Geography', axis=1), geo_df], axis=1) # -> concatenating the original DataFrame with the new one




In [124]:
#X = pd.get_dummies(X, columns=['Geography'])  automatically hot encodes the provided column

In [125]:
#Saving as pickle format for encoding Files
with open('/home/alkanol/Desktop/bankRunners/artifacts/Label_Encoder_Gender.pkl', 'wb') as f:
    pickle.dump(labEnco, f)
with open('/home/alkanol/Desktop/bankRunners/artifacts/OneHotEncoder_Geography.pkl', 'wb') as f:
    pickle.dump(OHE, f)


In [136]:
#Independent & Dependent Allocation
X=X
y=y
#As we have already performed operations on them and not on the csv_data itself
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2,random_state=42)
X_train

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
9254,686,1,32,6,0.00,2,1,1,179093.26,1.0,0.0,0.0
1561,632,1,42,4,119624.60,2,1,1,195978.86,0.0,1.0,0.0
1670,559,1,24,3,114739.92,1,1,0,85891.02,0.0,0.0,1.0
6087,561,0,27,9,135637.00,1,1,0,153080.40,1.0,0.0,0.0
6669,517,1,56,9,142147.32,1,0,0,39488.04,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
5734,768,1,54,8,69712.74,1,1,1,69381.05,1.0,0.0,0.0
5191,682,0,58,1,0.00,1,1,1,706.50,1.0,0.0,0.0
5390,735,0,38,1,0.00,3,0,0,92220.12,1.0,0.0,0.0
860,667,1,43,8,190227.46,1,1,0,97508.04,1.0,0.0,0.0


In [127]:
#Standardising_the_Data
scaler=StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)

with open("/home/alkanol/Desktop/bankRunners/artifacts/scaler.pkl",'wb') as f:
    pickle.dump(scaler,f)

### Using ANN

In [128]:
import tensorflow as tf 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime as dt 


In [129]:
model = Sequential(
    [
        Dense(64,activation='relu',input_shape=(X_train.shape[1],)),
        Dense(32,activation='relu'),
        Dense(1,activation='sigmoid'),
    ]
)
model.summary()

/home/alkanol/Desktop/bankRunners/.venv/lib/python3.11/site-packages/keras/src/layers/core/dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_12 (Dense)                │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [130]:
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [131]:
#Tensorboard
logs_dir = "/home/alkanol/Desktop/bankRunners/logs/fit/" + dt.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=logs_dir, histogram_freq=1)

In [132]:
#Early Stopping
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)


In [133]:
#Model_Train
ann_model = model.fit(
    X_train,
    y_train,
    validation_data=(X_test, y_test),
    epochs=100,
    batch_size=32,
    callbacks=[early_stopping, tensorboard_callback]
)


Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step - accuracy: 0.8056 - loss: 0.4581 - val_accuracy: 0.8330 - val_loss: 0.3940
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8441 - loss: 0.3831 - val_accuracy: 0.8570 - val_loss: 0.3573
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8564 - loss: 0.3542 - val_accuracy: 0.8505 - val_loss: 0.3490
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8589 - loss: 0.3457 - val_accuracy: 0.8575 - val_loss: 0.3443
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8618 - loss: 0.3390 - val_accuracy: 0.8525 - val_loss: 0.3465
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8605 - loss: 0.3363 - val_accuracy: 0.8575 - val_loss: 0.3445
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8606 - loss: 0.3337 - val_accuracy: 0.8590 - val_loss: 0.3436
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8634 - loss: 0.3302 - val_accu

In [134]:
model.save("/home/alkanol/Desktop/bankRunners/artifacts/ann_model.h5")

In [135]:
## Loading Tensorboard Extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [ ]:
%tensorboard --logdir /home/alkanol/Desktop/bankRunners/logs/fit